In [1]:
# ALS Baseline Recommender System
# Training, evaluation, and inference with cold start handling

import numpy as np
import pandas as pd
import pickle
import json
from scipy.sparse import load_npz, csr_matrix
import implicit
from sklearn.metrics import precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

import os
os.chdir("..")
print(os.getcwd())

/home/lenghanz/group-project-f25-shrekommender-system/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Libraries imported successfully
/home/lenghanz/group-project-f25-shrekommender-system


In [2]:
# Load the processed data
print("=== Loading Data ===")

# Load the training matrix
train_matrix = load_npz('data/processed/train_matrix.npz')
print(f"Training matrix shape: {train_matrix.shape}")
print(f"Training matrix sparsity: {100 * (1 - train_matrix.nnz / (train_matrix.shape[0] * train_matrix.shape[1])):.4f}%")

# Load mappings
with open('data/processed/user_mappings.pkl', 'rb') as f:
    user_mappings = pickle.load(f)
    
with open('data/processed/movie_mappings.pkl', 'rb') as f:
    movie_mappings = pickle.load(f)
    
# Load split metadata
with open('data/processed/split_metadata.pkl', 'rb') as f:
    split_metadata = pickle.load(f)

print(f"Users in training: {len(user_mappings['user_to_idx']):,}")
print(f"Movies in training: {len(movie_mappings['movie_to_idx']):,}")
print(f"Split strategy: {split_metadata['split_strategy']}")  # Fixed key name
print(f"Training interactions: {split_metadata['train_interactions']:,}")
print(f"Validation interactions: {split_metadata['val_interactions']:,}")
print("✓ Data loaded successfully")

=== Loading Data ===
Training matrix shape: (25262, 11557)
Training matrix sparsity: 99.9909%
Users in training: 25,262
Movies in training: 11,557
Split strategy: temporal split (last 20% of time)
Training interactions: 1,588,619
Validation interactions: 380,483
✓ Data loaded successfully


In [3]:
# Train ALS model
print("=== Training ALS Model ===")

# Model hyperparameters
factors = 64
iterations = 20
regularization = 0.01
alpha = 40

print("Model parameters:")
print(f"- Factors: {factors}")
print(f"- Iterations: {iterations}")
print(f"- Regularization: {regularization}")
print(f"- Alpha: {alpha}")

# Initialize and train the model
model = implicit.als.AlternatingLeastSquares(
    factors=factors,
    iterations=iterations,
    regularization=regularization,
    alpha=alpha,
    random_state=42
)

print("\nStarting training...")
# Train on the user-item matrix directly (NOT transposed)
# The implicit library expects user-item format for ALS
model.fit(train_matrix * alpha)
print("✓ Model training completed!")

# Get most popular movies for cold start handling
movie_popularity = np.array(train_matrix.sum(axis=0)).flatten()
popular_movie_indices = np.argsort(-movie_popularity)[:10]

print("\nMost popular movies (by interaction count):")
idx_to_movie = movie_mappings['idx_to_movie']
for i, idx in enumerate(popular_movie_indices):
    movie_name = idx_to_movie[idx]
    count = int(movie_popularity[idx])
    print(f"{i+1:2d}. {movie_name} (interactions: {count})")

=== Training ALS Model ===
Model parameters:
- Factors: 64
- Iterations: 20
- Regularization: 0.01
- Alpha: 40

Starting training...


100%|██████████| 20/20 [00:03<00:00,  6.02it/s]

✓ Model training completed!

Most popular movies (by interaction count):
 1. the+shawshank+redemption+1994 (interactions: 10417)
 2. inception+2010 (interactions: 9366)
 3. star+wars+1977 (interactions: 7334)
 4. interstellar+2014 (interactions: 5917)
 5. the+matrix+1999 (interactions: 5840)
 6. spirited+away+2001 (interactions: 5455)
 7. raiders+of+the+lost+ark+1981 (interactions: 5112)
 8. blade+runner+1982 (interactions: 5070)
 9. harry+potter+and+the+deathly+hallows+part+2+2011 (interactions: 4882)
10. the+lord+of+the+rings+the+fellowship+of+the+ring+2001 (interactions: 4779)


In [4]:
# Load validation data
print("=== Preparing Validation Data ===")

val_data = []
with open('data/processed/val_watch_segments.jsonl', 'r') as f:
    for line in f:
        val_data.append(json.loads(line))

print(f"Validation segments: {len(val_data):,}")

# Process validation data
val_users = []
val_movies = []
for segment in val_data:
    user_id = segment['user_id']
    movie_id = segment['movie_id']
    val_users.append(user_id)
    val_movies.append(movie_id)

# Count unique interactions
val_interactions = len(set(zip(val_users, val_movies)))
print(f"Validation interactions: {val_interactions:,}")
print(f"Validation users: {len(set(val_users)):,}")
print(f"Validation movies: {len(set(val_movies)):,}")

# Split validation data into known and new users
known_users = []
new_users = []
user_to_idx = user_mappings['user_to_idx']
movie_to_idx = movie_mappings['movie_to_idx']

for user_id, movie_id in zip(val_users, val_movies):
    if user_id in user_to_idx and movie_id in movie_to_idx:
        known_users.append((user_id, movie_id))
    else:
        new_users.append((user_id, movie_id))

print(f"\nValidation split:")
print(f"Known users: {len(known_users):,} interactions")
print(f"New users (cold start): {len(new_users):,} interactions")

=== Preparing Validation Data ===
Validation segments: 56,723
Validation interactions: 3,405
Validation users: 3,392
Validation movies: 2,338

Validation split:
Known users: 11,680 interactions
New users (cold start): 45,043 interactions


In [5]:
# Define evaluation functions
import time

def evaluate_recommendations(recommendations, ground_truth, k=10):
    """Evaluate recommendation quality using precision, recall, and F1."""
    recommendations = recommendations[:k]
    
    if len(ground_truth) == 0:
        return 0, 0, 0
    
    hits = len(set(recommendations) & set(ground_truth))
    
    precision = hits / len(recommendations) if recommendations else 0
    recall = hits / len(ground_truth) if ground_truth else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Track inference times
inference_times = []

def get_als_recommendations(user_idx, model, train_matrix, n_items=10):
    """Get ALS recommendations for a user.
    
    Following the production code pattern from als_recommender.py
    """
    # Get recommendations directly - model was trained on user-item matrix
    # model.recommend returns (indices, scores) as two separate arrays
    
    # Record inference time
    start_time = time.time()
    
    recommended_indices, scores = model.recommend(
        userid=user_idx,
        user_items=train_matrix[user_idx],  # Pass the user's row
        N=n_items,
        filter_already_liked_items=True
    )
    
    inference_time = time.time() - start_time
    inference_times.append(inference_time)
    
    return recommended_indices  # Return just the indices

print("✓ Evaluation functions defined")

✓ Evaluation functions defined


In [6]:
# Evaluate on known users
print("=== Evaluating Model on Known Users ===")

# Reset inference times for this evaluation
inference_times.clear()

# Sample known users for evaluation
known_user_interactions = {}
for user_id, movie_id in known_users:
    if user_id not in known_user_interactions:
        known_user_interactions[user_id] = []
    known_user_interactions[user_id].append(movie_id)

# Evaluate a sample of known users
sample_size = min(100, len(known_user_interactions))
sample_users = list(known_user_interactions.keys())[:sample_size]

known_precisions = []  # Changed variable name for clarity
known_recalls = []
known_f1_scores = []
known_hit_rate = 0

print_samples = 5
samples_printed = 0

for user_id in sample_users:
    user_idx = user_to_idx[user_id]
    
    # Get recommendations (pass train_matrix as parameter)
    rec_indices = get_als_recommendations(user_idx, model, train_matrix, n_items=20)
    rec_movies = [idx_to_movie[idx] for idx in rec_indices]
    
    # Get ground truth
    ground_truth = known_user_interactions[user_id]
    
    # Evaluate
    precision, recall, f1 = evaluate_recommendations(rec_movies, ground_truth, k=20)
    known_precisions.append(precision)
    known_recalls.append(recall)
    known_f1_scores.append(f1)
    
    if precision > 0:
        known_hit_rate += 1
    
    # Print samples
    if samples_printed < print_samples:
        status = "✓" if precision > 0 else "✗"
        print(f"\n{status} User {user_id}: True={ground_truth[0] if ground_truth else 'none'}")
        print(f"   Recommended: {', '.join(rec_movies[:5])}")
        samples_printed += 1

known_hit_rate = known_hit_rate / sample_size

print(f"\nKnown Users Performance (N=10):")
print(f"- Average Precision: {np.mean(known_precisions):.4f}")
print(f"- Average Recall: {np.mean(known_recalls):.4f}")
print(f"- Average F1: {np.mean(known_f1_scores):.4f}")
print(f"- Hit Rate: {known_hit_rate:.4f}")
print(f"- Evaluated on {sample_size} users")

# Report inference time statistics
if inference_times:
    print(f"\nInference Time Statistics:")
    print(f"- Mean: {np.mean(inference_times)*1000:.2f} ms")
    print(f"- Median: {np.median(inference_times)*1000:.2f} ms")
    print(f"- Min: {np.min(inference_times)*1000:.2f} ms")
    print(f"- Max: {np.max(inference_times)*1000:.2f} ms")
    print(f"- Total inference time: {sum(inference_times):.3f} seconds for {len(inference_times)} recommendations")

=== Evaluating Model on Known Users ===

✗ User 47527: True=harry+potter+and+the+deathly+hallows+part+2+2011
   Recommended: the+shawshank+redemption+1994, nausica+of+the+valley+of+the+wind+1984, star+wars+1977, 2001+a+space+odyssey+1968, scent+of+a+woman+1992

✗ User 83115: True=the+cobweb+1955
   Recommended: the+shawshank+redemption+1994, 2001+a+space+odyssey+1968, nausica+of+the+valley+of+the+wind+1984, fight+club+1999, mickey_+donald_+goofy+the+three+musketeers+2004

✗ User 34100: True=sabotage+1936
   Recommended: the+dark+knight+2008, aladdin+1992, snow+white+and+the+seven+dwarfs+1937, harry+potter+and+the+order+of+the+phoenix+2007, the+living+daylights+1987

✗ User 137263: True=i+bought+a+vampire+motorcycle+1990
   Recommended: the+shawshank+redemption+1994, star+wars+1977, the+godfather+1972, monsters_+inc.+2001, inception+2010

✗ User 66688: True=black+and+white+in+color+1976
   Recommended: winnie+the+pooh+and+the+honey+tree+1966, harry+potter+and+the+chamber+of+secrets+2002

In [7]:
# Evaluate cold start strategy (most popular items)
print("=== Evaluating Cold Start Strategy ===")

# Get most popular movies for recommendations
movie_popularity = np.array(train_matrix.sum(axis=0)).flatten()
popular_movie_indices = np.argsort(-movie_popularity)[:20]
popular_movies = [idx_to_movie[idx] for idx in popular_movie_indices]

print("Most popular recommendations for new users:")
for i, movie in enumerate(popular_movies):
    print(f"{i+1:2d}. {movie}")

# Evaluate on new users
new_user_interactions = {}
for user_id, movie_id in new_users:
    if user_id not in new_user_interactions:
        new_user_interactions[user_id] = []
    new_user_interactions[user_id].append(movie_id)

# Sample new users for evaluation
sample_size = min(50, len(new_user_interactions))
sample_new_users = list(new_user_interactions.keys())[:sample_size]

new_precisions = []
new_recalls = []
new_f1_scores = []
new_hit_rate = 0

for user_id in sample_new_users:
    # Use popular movies as recommendations
    rec_movies = popular_movies
    
    # Get ground truth
    ground_truth = new_user_interactions[user_id]
    
    # Evaluate
    precision, recall, f1 = evaluate_recommendations(rec_movies, ground_truth, k=20)
    new_precisions.append(precision)
    new_recalls.append(recall)
    new_f1_scores.append(f1)
    
    if precision > 0:
        new_hit_rate += 1

new_hit_rate = new_hit_rate / sample_size

print(f"\nNew Users Performance (Most Popular Strategy, N=10):")
print(f"- Average Precision: {np.mean(new_precisions):.4f}")
print(f"- Average Recall: {np.mean(new_recalls):.4f}")
print(f"- Average F1: {np.mean(new_f1_scores):.4f}")
print(f"- Hit Rate: {new_hit_rate:.4f}")
print(f"- Evaluated on {sample_size} users")

print("\n" + "="*50)
print("EVALUATION SUMMARY")
print("="*50)
print(f"Known Users (ALS): Hit Rate = {known_hit_rate:.4f}, F1 = {np.mean(known_f1_scores):.4f}")
print(f"New Users (Popular): Hit Rate = {new_hit_rate:.4f}, F1 = {np.mean(new_f1_scores):.4f}")
print(f"Overall Hit Rate: {(known_hit_rate + new_hit_rate) / 2:.4f}")

=== Evaluating Cold Start Strategy ===
Most popular recommendations for new users:
 1. the+shawshank+redemption+1994
 2. inception+2010
 3. star+wars+1977
 4. interstellar+2014
 5. the+matrix+1999
 6. spirited+away+2001
 7. raiders+of+the+lost+ark+1981
 8. blade+runner+1982
 9. harry+potter+and+the+deathly+hallows+part+2+2011
10. the+lord+of+the+rings+the+fellowship+of+the+ring+2001
11. the+dark+knight+2008
12. fight+club+1999
13. the+lord+of+the+rings+the+two+towers+2002
14. monsters_+inc.+2001
15. forrest+gump+1994
16. nausica+of+the+valley+of+the+wind+1984
17. the+fifth+element+1997
18. harry+potter+and+the+chamber+of+secrets+2002
19. my+neighbor+totoro+1988
20. whiplash+2014

New Users Performance (Most Popular Strategy, N=10):
- Average Precision: 0.0010
- Average Recall: 0.0006
- Average F1: 0.0008
- Hit Rate: 0.0200
- Evaluated on 50 users

EVALUATION SUMMARY
Known Users (ALS): Hit Rate = 0.0300, F1 = 0.0012
New Users (Popular): Hit Rate = 0.0200, F1 = 0.0008
Overall Hit Rate: 0

In [8]:
# Create a recommendation system class
class ALSRecommendationSystem:
    def __init__(self, model, train_matrix, user_mappings, movie_mappings, popular_movies):
        self.model = model
        self.train_matrix = train_matrix
        self.user_to_idx = user_mappings['user_to_idx']
        self.idx_to_user = user_mappings['idx_to_user']
        self.movie_to_idx = movie_mappings['movie_to_idx']
        self.idx_to_movie = movie_mappings['idx_to_movie']
        self.popular_movies = popular_movies
        
    def recommend_for_user(self, user_id, n_recommendations=10):
        """Get recommendations for a user (handles cold start).
        
        Following the production code pattern from als_recommender.py
        """
        if user_id in self.user_to_idx:
            # Known user - use ALS
            user_idx = self.user_to_idx[user_id]
            
            # Get recommendations (following production code lines 181-186)
            # model.recommend returns (indices, scores) as two separate arrays
            recommended_indices, scores = self.model.recommend(
                userid=user_idx,
                user_items=self.train_matrix[user_idx],
                N=n_recommendations,
                filter_already_liked_items=True
            )
            
            # Combine indices and scores
            recommendations = []
            for idx, score in zip(recommended_indices, scores):
                movie_id = self.idx_to_movie[idx]
                recommendations.append((movie_id, float(score)))
            
            return {
                'user_id': user_id,
                'type': 'known, ALS',
                'recommendations': recommendations
            }
        else:
            # New user - return most popular
            return {
                'user_id': user_id,
                'type': 'new, most_popular',
                'recommendations': [(movie, None) for movie in self.popular_movies[:n_recommendations]]
            }
    
    def find_similar_movies(self, movie_id, n_similar=10):
        """Find movies similar to a given movie."""
        if movie_id not in self.movie_to_idx:
            return None
        
        movie_idx = self.movie_to_idx[movie_id]
        # similar_items also returns (indices, scores) as two separate arrays
        similar_indices, scores = self.model.similar_items(movie_idx, N=n_similar+1)
        
        # Skip the first result (the movie itself) and combine indices with scores
        similar_movies = []
        for idx, score in zip(similar_indices[1:], scores[1:]):
            movie_name = self.idx_to_movie[idx]
            similar_movies.append((movie_name, float(score)))
        
        return {
            'movie_id': movie_id,
            'similar_movies': similar_movies
        }

# Initialize the recommendation system
recommender = ALSRecommendationSystem(
    model=model,
    train_matrix=train_matrix,
    user_mappings=user_mappings,
    movie_mappings=movie_mappings,
    popular_movies=popular_movies
)

print("=== Recommendation System API ===")
print("✓ Recommendation System initialized")

=== Recommendation System API ===
✓ Recommendation System initialized


In [9]:
# Demo the recommendation system
print("=== Recommendation System Demo ===")
print("1. Testing with known users:")

# Test with some known users
test_users = [100006, 100013, 10002]
for user_id in test_users:
    result = recommender.recommend_for_user(user_id, n_recommendations=5)
    print(f"\nUser: {result['user_id']} ({result['type']})")
    print("Recommendations:")
    for i, (movie, score) in enumerate(result['recommendations'], 1):
        if score:
            print(f"  {i}. {movie} (score: {score:.3f})")
        else:
            print(f"  {i}. {movie}")

# Test with a new user
print("\n2. Testing with new user:")
new_user_id = "new_user_12345"
result = recommender.recommend_for_user(new_user_id, n_recommendations=5)
print(f"\nUser: {result['user_id']} ({result['type']})")
print("Recommendations:")
for i, (movie, score) in enumerate(result['recommendations'], 1):
    if score:
        print(f"  {i}. {movie} (score: {score:.3f})")
    else:
        print(f"  {i}. {movie} ")

# Test movie similarity
print("\n3. Testing similar movies:")
test_movie = "+nos+amours+1983"
similar_result = recommender.find_similar_movies(test_movie, n_similar=5)
if similar_result:
    print(f"\nMovies similar to: {similar_result['movie_id']}")
    for i, (movie, score) in enumerate(similar_result['similar_movies'], 1):
        print(f"  {i}. {movie} (similarity: {score:.3f})")

print("\n" + "="*50)
print("🎬 ALS RECOMMENDATION SYSTEM READY!")
print("="*50)
print("Features:")
print("✓ Personalized recommendations for known users (ALS)")
print("✓ Most popular recommendations for new users")
print("✓ Movie similarity search")
print("✓ Comprehensive evaluation metrics")
print(f"\nSystem Statistics:")
print(f"- Training users: {len(user_to_idx):,}")
print(f"- Training movies: {len(movie_to_idx):,}")
print(f"- Model factors: {factors}")
print(f"- Training iterations: {iterations}")

# Save the model and recommendation system
print("\n=== Saving Model ===")
import os
os.makedirs('data/models', exist_ok=True)

# Save the complete recommendation system
with open('data/models/als_model.pkl', 'wb') as f:
    pickle.dump({
        'model': model,
        'train_matrix': train_matrix,
        'user_mappings': user_mappings,
        'movie_mappings': movie_mappings,
        'popular_movies': popular_movies,
        'hyperparameters': {
            'factors': factors,
            'iterations': iterations,
            'regularization': regularization,
            'alpha': alpha
        }
    }, f)

print("✓ Model saved to: data/models/als_model.pkl")
print("Ready for production deployment!")

=== Recommendation System Demo ===
1. Testing with known users:

User: 100006 (new, most_popular)
Recommendations:
  1. the+shawshank+redemption+1994
  2. inception+2010
  3. star+wars+1977
  4. interstellar+2014
  5. the+matrix+1999

User: 100013 (new, most_popular)
Recommendations:
  1. the+shawshank+redemption+1994
  2. inception+2010
  3. star+wars+1977
  4. interstellar+2014
  5. the+matrix+1999

User: 10002 (new, most_popular)
Recommendations:
  1. the+shawshank+redemption+1994
  2. inception+2010
  3. star+wars+1977
  4. interstellar+2014
  5. the+matrix+1999

2. Testing with new user:

User: new_user_12345 (new, most_popular)
Recommendations:
  1. the+shawshank+redemption+1994 
  2. inception+2010 
  3. star+wars+1977 
  4. interstellar+2014 
  5. the+matrix+1999 

3. Testing similar movies:

Movies similar to: +nos+amours+1983
  1. grand+canyon+1991 (similarity: 1.000)
  2. why+we+fight+the+nazis+strike+1943 (similarity: 1.000)
  3. the+tree+2010 (similarity: 1.000)
  4. eagle